<a href="https://colab.research.google.com/github/Penitto/risk_project1/blob/master/Iriski.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Выделенные риск-факторы

1. Фондовый риск — риск снижения цены акций; __индекс РТС и МОЕКС__
2. Процентный риск — риск изменения процентных ставок; __процентные ставки__
3. Валютный риск — риск изменения курсов валют; __курс доллара и курс юаня__
4. Товарный риск — риск изменения цен товаров; __цены на нефть__


# Стохастическая модель динамики 

In [2]:
import numpy as np
import pandas as pd
import os
from scipy.interpolate import CubicSpline

In [79]:
class Portfolio():
    def __init__(self, bonds, shares, cur, risk):
        self.bonds = bonds
        self.shares = shares
        self.cur = cur
        self.risk = risk
        self.price = 0
        
    # Посчитать цену портфеля в определённый день
    def countPriceInDate(self, date):
        res = 0
        
        for i in self.bonds:
            res += i[3] * i[1]['<CLOSE>'].loc[date]
            
        for i in self.shares:
            res += i[3] * i[1]['<CLOSE>'].loc[date]
            
        
    # Посчитать объём портфеля в уе
    def countInitialValue(self):
        for i in range(len(self.bonds)):
            amount = self.bonds[i][2] / self.bonds[i][1]['<CLOSE>'][0]
            self.bonds[i] = (*self.bonds[i], amount)
            
        for i in range(len(self.shares)):
            amount = self.shares[i][2] / self.shares[i][1]['<CLOSE>'][0]
            self.shares[i] = (*self.shares[i], amount)
            
        for i in range(len(self.cur)):
            amount = self.cur[i][2] / self.cur[i][1]['<CLOSE>'][0]
            self.cur[i] = (*self.cur[i], amount)

In [200]:
# Загрузка данных в класс
shares = ['./shares/AFLT_160101_200101.csv', 
          './shares/GAZP_160101_200101.csv',
          './shares/GMKN_160101_200101.csv', 
          './shares/KMAZ_160101_200101.csv', 
          './shares/LKOH_160101_200101.csv', 
          './shares/PIKK_160101_200101.csv', 
          './shares/MGNT_160101_200101.csv', 
          './shares/RBCM_160101_200101.csv', 
          './shares/ROSN_160101_200101.csv', 
          './shares/SBER_160101_200101.csv']

shares_name = [i[9:13] for i in shares]

# Нужно интерполировать
bonds = ['./bonds/SU26212RMFS9_160101_200101.csv',
         './bonds/SU26205RMFS3_160101_200101.csv',
         './bonds/SU26207RMFS9_160101_200101.csv', 
         './bonds/SU26209RMFS5_160101_200101.csv', 
         './bonds/SU26211RMFS1_160101_200101.csv']

bonds_name = [i[8:20] for i in bonds]

currencies = ['./index/USDCB_160101_200101.csv', './index/CNYCB_160101_200101.csv']

days = 1010

In [201]:
risk_df = pd.read_csv(shares[0], index_col='<DATE>').drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1)
risk_df.index = pd.to_datetime(risk_df.index)
k = 1
for i in shares[1:]:
    tmp = pd.read_csv(i, index_col='<DATE>') \
            .drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1) \
            .rename(columns={'<CLOSE>': shares_name[k]})
    tmp.index = pd.to_datetime(tmp.index)
    risk_df = risk_df.join(tmp, how='left')
    k += 1
    
k = 0
for i in bonds:
    tmp = pd.read_csv(i, index_col='<DATE>') \
            .drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1) \
            .rename(columns={'<CLOSE>' : bonds_name[k]})
    tmp.index = pd.to_datetime(tmp.index)
    risk_df = risk_df.join(tmp, how='left')
    k += 1

risk_df = risk_df.rename(columns={'<CLOSE>' : shares_name[0]})

In [202]:
risk_df

,AFLT,GAZP,GMKN,KMAZ,LKOH,PIKK,MGNT,RBCM,ROSN,SBER,SU26212RMFS9,SU26205RMFS3,SU26207RMFS9,SU26209RMFS5,SU26211RMFS1
<DATE>,,,,,,,,,,,,,,,
2016-04-01,56.20,134.91,8922.0,36.10,2310.2,215.5,11366.0,4.730,251.00,98.01,83.9998,92.3500,91.8999,NaN,87.3998
2016-05-01,54.45,136.45,9085.0,36.05,2380.0,213.2,11350.0,4.765,253.20,98.89,84.1000,92.3500,91.9995,91.3000,87.6996
2016-06-01,54.05,135.94,8990.0,36.15,2378.0,212.5,11320.0,4.705,252.40,96.85,83.7494,92.0999,91.5999,91.2000,87.6999
2016-11-01,52.06,131.38,8600.0,35.60,2257.9,208.9,10667.0,4.595,241.05,91.03,80.8000,90.4499,89.2999,88.7000,85.5007
2016-12-01,53.25,133.54,8553.0,35.40,2223.5,210.1,10367.0,4.540,239.90,91.71,79.6579,89.2510,87.8001,87.5011,84.1501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,103.08,254.87,19630.0,56.80,6080.5,395.0,3371.5,3.288,451.40,248.67,105.4930,102.6980,111.5200,104.5860,103.7560
2019-12-25,103.34,253.25,19890.0,57.30,6081.0,394.4,3416.0,3.500,454.00,248.04,105.5470,102.8640,111.4570,104.5600,104.0680
2019-12-26,103.94,254.10,19220.0,57.90,6085.0,398.1,3449.0,3.446,452.50,248.24,105.5800,102.7130,111.6010,104.6400,103.9990


In [166]:
port = Portfolio(bonds, shares, currencies)
port.countInitialValue()

,<CLOSE>_1,<CLOSE>_2,<CLOSE>_3,<CLOSE>_4,<CLOSE>_5
<DATE>,,,,,
2016-04-01,83.9998,92.3500,91.8999,NaN,87.3998
2016-05-01,84.1000,92.3500,91.9995,91.3000,87.6996
2016-06-01,83.7494,92.0999,91.5999,91.2000,87.6999
2016-11-01,80.8000,90.4499,89.2999,88.7000,85.5007
2016-12-01,79.6579,89.2510,87.8001,87.5011,84.1501
...,...,...,...,...,...
2019-12-24,105.4930,102.6980,111.5200,104.5860,103.7560
2019-12-25,105.5470,102.8640,111.4570,104.5600,104.0680
2019-12-26,105.5800,102.7130,111.6010,104.6400,103.9990


In [167]:
for i in port.cur:
    print(len(i[1]))

989
304


In [11]:
def get_index():
    moex_ind = pd.read_csv('./index/IMOEX_160101_200101.csv')
    rts_ind = pd.read_csv('./index/RTSI_160101_200101.csv')
    zerobond = pd.read_csv('./zerobond.csv', sep=';')
    
    weeks_in_month = (365/7)*(1/12)
    maturity_rub = [3,6,9,12,24,36,60,84,120,180,240,360,]
    maturity_rub= [i * weeks_in_month for i in maturity_rub]
    
    interpolate_rub = CubicSpline(maturity_rub, ds_rate_rub)
    
    interval_rub = np.arange(0, 54, 2)
    
    df_rub_int = pd.DataFrame(data={'maturity_rub_2weeks': interval_rub, 'rub_act': interpolate_rub(interval_rub)})
    
    df_rub_usd_int=pd.concat([df_usd_int, df_rub_int], axis=1, sort=False)
    df_rub_usd_int=df_rub_usd_int.drop(['maturity_rub_2weeks'], axis=1)
    df_rub_usd_int=df_rub_usd_int.rename(columns={"maturity_usd_2weeks":"maturity"})
    df_rub_usd_int['maturity_frac'] = df_rub_usd_int['maturity']/54

    # s=0.0134
    s=1/0.01442

    new_rates = pd.concat(
        [
            df_rub_usd_int,
            df_rub_usd_int.diff(1).rename(columns={x:x.replace('act','diff') for x in df_rub_usd_int.columns})
        ],
        axis=1)

    new_rates.fillna(0, inplace=True)
    new_rates['fx_act']=(s*(1+new_rates['usd_act']*0.01)/(1+new_rates['rub_act']*0.01))
    new_rates['fx_diff'] =new_rates['fx_act'].diff()


    curve_rub_act = new_rates.loc[1:,'rub_act']
    curve_usd_act = new_rates.loc[1:,'usd_act']
    curve_fx_act = new_rates.loc[1:,'fx_act']

    curve_rub_diff = new_rates.loc[1:,'rub_diff']
    curve_usd_diff = new_rates.loc[1:,'usd_diff']
    curve_fx_diff = new_rates.loc[1:,'fx_diff']
    init = new_rates.loc[0,['rub_act','usd_act','fx_act']]


    return  (
        curve_rub_act,
        curve_usd_act,
        curve_fx_act,
        curve_rub_diff,
        curve_usd_diff,
        curve_fx_diff,
        init)
    
    
    
    
    def stoch_wrapper(decomp):
        def make_stoch(num):
            sigma=[0.03, 0.0093, 0.11]
            stoch_generator = np.dot(np.random.normal(size=(num,3)),decomp)*sigma
            return stoch_generator
        return make_stoch

    
stoch_generator = stoch_wrapper(get_decomp())

def simulate_hull_white(
    sim_number = 10,):
    rub_alpha=0.03
    sigma=[0.03, 0.0093, 0.11]
    k_fx=0.015
    dt=14/365
    timesteps = 26

    (
        curve_rub,
        curve_rub_df,
        init
        ) = get_rates()


    results = np.zeros(shape=(timesteps+1, 3, sim_number))

    passed_time=0

    for sim_ix in range(sim_number):
        results[0, :, sim_ix] = init
        stochs = stoch_generator(timesteps+1)
        for i, (rate_rub, df_rub, stoch_tuple) in enumerate(zip(curve_rub,curve_rub_df, stochs)):
            
            passed_time += dt

            theta_rub = df_rub + rub_alpha * rate_rub + (sigma[0]**2) * (1 - np.exp(-2 * rub_alpha * passed_time)) / 2 * rub_alpha

            results[i + 1, 0, sim_ix] = (theta_rub - rub_alpha* results[:, 0, sim_ix].sum()) * dt + stoch_tuple[0]

    return results

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,RTSI,D,04/01/16,00:00:00,755.16,761.68,741.67,749.28,204806463
1,RTSI,D,05/01/16,00:00:00,747.94,753.97,743.42,752.70,242929972
2,RTSI,D,06/01/16,00:00:00,753.06,754.04,735.62,736.82,210962101
3,RTSI,D,11/01/16,00:00:00,721.68,721.68,699.13,699.13,0
4,RTSI,D,12/01/16,00:00:00,689.33,706.31,681.46,695.32,0


# Оценка справедливой стоимости в зависимости от риск-факторов

# Оценка риска по портфелю

# Простая количественная валидация